##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [146]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Detect landmarks in 2D vid and store them

In [147]:
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def detect_landmark_video(vid_path,out_path,start_frame,num_frames):
    # Use OpenCV’s VideoCapture to load the input video:
    video_capture = cv2.VideoCapture(vid_path)

    # Check if the video is opened successfully
    if not video_capture.isOpened():
        print(f"Error: Could not open video {vid_path}.")
        return
    
    # Set the starting frame
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # counter of frames:
    frame_count = 0

    annotated_results = []
    # Create a hand landmarker instance with the video mode:
    with mp_hands.Hands(
    model_complexity=1,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
      while video_capture.isOpened():
        ret, image = video_capture.read()
        if not ret:
          # When vid is done:
          break
        
        if frame_count < num_frames:
          frame_count += 1
          # To improve performance, optionally mark the image as not writeable to
          # pass by reference.
          image.flags.writeable = False
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          results = hands.process(image)

          # Draw the hand annotations on the image.
          image.flags.writeable = True
          image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
          if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
              mp_drawing.draw_landmarks(
                  image,
                  hand_landmarks,
                  mp_hands.HAND_CONNECTIONS,
                  mp_drawing_styles.get_default_hand_landmarks_style(),
                  mp_drawing_styles.get_default_hand_connections_style())
          
          image_height, image_width, _ = image.shape
          # append the annotattions to the list:
          annotated_results.append(results)
        else:
           # if reached the ending frame:
           break

      # going through frames:
      annotations = []
      for result in annotated_results:
        x_points = []
        y_points = []
        # going through landmarks of the frame:
        for cnt, hand_landmark in enumerate(result.multi_hand_landmarks[0].landmark):
          x_points.append(hand_landmark.x * image_width)
          y_points.append(hand_landmark.y * image_height)
        x_points.extend(y_points)
        #print(x_points)
        annotations.append(x_points)
      

      # Define dataframe to hold the dannotations ata:
      # Define the number of columns:
      num_columns = 21

      # Create column names:
      columns = [f'x{i}' for i in range(0, num_columns)] + [f'y{i}' for i in range(0, num_columns)]

      # Create the DataFrame
      df = pd.DataFrame(annotations,columns=columns)
          
      #print(df)
    
    # Release the VideoCapture object:
    video_capture.release()
          
        

In [148]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def make_2D_reconstruction(vid_path,out_path,start_frame,num_frames):
    # Use OpenCV’s VideoCapture to load the input video:
    video_capture = cv2.VideoCapture(vid_path)

    # Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS:
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)

    # Create video writer instance to make an annotated vid:
    ret, image = video_capture.read()
    height, width, _ = image.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_path, fourcc, frame_rate, (width, height))

    annotated_results = []
    # Create a hand landmarker instance with the video mode:
    with mp_hands.Hands(
    model_complexity=1,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
      while video_capture.isOpened():
        ret, image = video_capture.read()
        if not ret:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          break
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
        
        # append the annotattions to the list:
        annotated_results.append(results)

        # write the annotated vid:
        out.write(image)
      
      # Release the VideoCapture object:
      video_capture.release()
      out.release()
          

      

## 2D prediction

In [149]:
import os
import time

data_path = '/media/pruszynski/Data'
recording_name = 'Recording_2023-05-15T154223' #Jon's no board
# recording_name = 'Recording_2023-05-05T100719' #Tomo's messy situation
# recording_name = 'Recording_2024-01-31T114025' # Ali's normal recording
pred_path = './Predictions2D'

start_frame = 0
num_frames = 100
# Loop through all .avi vids in the folder
for filename in os.listdir(os.path.join(data_path, recording_name)):
    start_time = time.time()  # Record the start time of the iteration

    if filename.endswith('.avi'):
        # Construct the full path to the .avi file and output prediction:
        vid_path = os.path.join(data_path, recording_name, filename)
        out_path = os.path.join(pred_path, recording_name, filename)

        # Ensure the prediction directory exists, create it if it doesn't
        if not os.path.exists(os.path.join(pred_path,recording_name)):
            os.makedirs(os.path.join(pred_path,recording_name))
        
        # make 2D predictions:
        detect_landmark_video(vid_path,out_path,start_frame,num_frames)
    
    end_time = time.time()  # Record the end time of the iteration
    iteration_time = end_time - start_time  # Calculate the time taken by the iteration
    print(f"Duration: {iteration_time:.2f} seconds")


I0000 00:00:1715381306.339154  926072 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715381306.397248  963006 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 525.105.17), renderer: NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2
/home/pruszynski/Desktop/Projects/HandTracker3D/.conda/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Duration: 3.24 seconds


I0000 00:00:1715381309.581366  926072 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715381309.697241  963025 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 525.105.17), renderer: NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2


Duration: 3.22 seconds


I0000 00:00:1715381312.800244  926072 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715381312.914194  963042 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 525.105.17), renderer: NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2


TypeError: 'NoneType' object is not subscriptable